In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

query = """
 SELECT 
 token_address as addr,
 value as val,
 block_timestamp as time
FROM
`bigquery-public-data.crypto_ethereum.token_transfers` 

WHERE date(block_timestamp)>="2021-01-22"
LIMIT 1000000
"""

In [ ]:


query_job = client.query(query)

iterator = query_job.result(timeout=30)
rows = list(iterator)

# Transform the rows into a nice pandas dataframe
df = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))

# Look at the first 15
df.head(15)


In [ ]:
#show the token_address with the most transactions
listtop5=df.addr.value_counts()[:5].index.tolist()
print(listtop5)#print the top 5 most transacted coins


In [ ]:
client = bigquery.Client()
#get the token tiker and the decimals
query = """
 SELECT 
 address as addr,
 decimals as deci,
 name as name
FROM
`bigquery-public-data.crypto_ethereum.tokens` 
"""

query_job = client.query(query)

iterator = query_job.result(timeout=30)
rows2 = list(iterator)
#print (rows)
# Transform the rows into a nice pandas dataframe
df2 = pd.DataFrame(data=[list(x.values()) for x in rows2], columns=list(rows2[0].keys()))

# Look at the first 10
df2.head(10)

In [ ]:
#check the top5 coins and extract the decimals + tiker for them
df['val'] = df['val'].astype(float)

digit=[0]*5 #generate two arrays
name=[0]*5

digit[0]=df2.loc[df2['addr'] == listtop5[0]]['deci'].values #extract decimals and name for all top5 tokens
name[0]=df2.loc[df2['addr'] == listtop5[0]]['name'].values

digit[1]=df2.loc[df2['addr'] == listtop5[1]]['deci'].values
name[1]=df2.loc[df2['addr'] == listtop5[1]]['name'].values

digit[2]=df2.loc[df2['addr'] == listtop5[2]]['deci'].values
name[2]=df2.loc[df2['addr'] == listtop5[2]]['name'].values

digit[3]=df2.loc[df2['addr'] == listtop5[3]]['deci'].values
name[3]=df2.loc[df2['addr'] == listtop5[3]]['name'].values

digit[4]=df2.loc[df2['addr'] == listtop5[4]]['deci'].values
name[4]=df2.loc[df2['addr'] == listtop5[4]]['name'].values

for x in range(0, 5): #check for not listed tokens
    if listtop5[x]== "0x6b175474e89094c44da98b954eedeac495271d0f": #DAI
        digit[x]=18
        name[x]=["Dai Stablecoin"]
    if listtop5[x]== "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48": # USD
        digit[x]=6
        name[x]=["USD Coin"]
#maybe there are some more tokens who need to be added
        
print(digit)
print(name)

In [ ]:
#get the total volume of the top5 coins
import math
vol0=df.loc[df['addr'] == listtop5[0], 'val'].sum() #collect all val in df when addr=var
vol1=df.loc[df['addr'] == listtop5[1], 'val'].sum()
vol2=df.loc[df['addr'] == listtop5[2], 'val'].sum()
vol3=df.loc[df['addr'] == listtop5[3], 'val'].sum()
vol4=df.loc[df['addr'] == listtop5[4], 'val'].sum()

#get the correct volume
vol0=vol0/math.pow(10,digit[0])
vol1=vol1/math.pow(10,digit[1])
vol2=vol2/math.pow(10,digit[2])
vol3=vol3/math.pow(10,digit[3])
vol4=vol4/math.pow(10,digit[4])


print(name[0],vol0,name[1],vol1,name[2],vol2,name[3],vol3,name[4],vol4)

In [ ]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

#here we could reduce the data to collect
#maybe there is a way to take the top add to the query
query = """
 SELECT 
 token_address as addr,
 value as val,
 date(block_timestamp) as time
FROM
`bigquery-public-data.crypto_ethereum.token_transfers` 

WHERE date(block_timestamp)>="2021-01-20"
LIMIT 10000000
"""

In [ ]:
query_job = client.query(query)

iterator = query_job.result(timeout=30)
rows = list(iterator)

# Transform the rows into a nice pandas dataframe
df3 = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))

# Look at the first 15
df3.head(15)

In [ ]:
import datetime

df3['time'] = pd.to_datetime(df3['time']) #convert the time format
qwert=[]
token0=pd.DataFrame(qwert, columns=['day', 'transac'])#generat a empty dataframe

for x in range(0, 30): #move one month and check the transactions for a token
    d=datetime.date.today() - datetime.timedelta(days=x) #generate the date
    d=d.strftime("%Y-%m-%d") #edit it in the correct format
    a=df3[(df3["addr"] == listtop5[0]) & (df3["time"]==d)].count() #collect all transactions for this data
    a=a[0] #filter it
    token0.loc[x] =[d,a] #add data to the frame
print(name[0])
print (token0)

In [ ]:
qwert=[]
token1=pd.DataFrame(qwert, columns=['day', 'transac'])#generat a empty dataframe

for x in range(0, 30): #move one month and check the transactions for a token
    d=datetime.date.today() - datetime.timedelta(days=x) #generate the date
    d=d.strftime("%Y-%m-%d") #edit it in the correct format
    a=df3[(df3["addr"] == listtop5[1]) & (df3["time"]==d)].count()#collect all transactions for this data
    a=a[0]#filter it
    token1.loc[x] =[d,a]#add data to the frame
print(name[1])
print (token1)